In [1]:
import requests
import pandas as pd

URLS

In [2]:
def get_point_url(base_url, lon, lat):
    return base_url + f'/api/category/pmp3g/version/2/geotype/point/lon/{lon}/lat/{lat}/data.json'

In [3]:
base_url = 'https://opendata-download-metfcst.smhi.se'
params_url = '/api/category/pmp3g/version/2/parameter.json'
valid_times_url = '/api/category/pmp3g/version/2/validtime.json'

Get parameters

In [4]:
parameter_response = requests.get(base_url+params_url).json()['parameter']

parameter_dict = {}
parameter_list = []

for element in parameter_response:
    parameter_list.append(element['name'])
    parameter_dict[element['name']] = element


Get locations with Name, Latitud, Longitud and Höjd (m)

In [5]:
locations_df = pd.read_csv('smhi_locations.csv', sep=';')
locations_df = locations_df.where(locations_df['Aktiv']=='Ja').dropna().drop(columns=['Id', 'Aktiv'])

In [6]:
selected_locations = ['Stockholm']

In [7]:
valid_times = requests.get(base_url+valid_times_url).json()

In [8]:
lat = locations_df.where(locations_df['Namn'] == 'Stockholm').dropna()['Latitud'].values[0]
lon = locations_df.where(locations_df['Namn'] == 'Stockholm').dropna()['Longitud'].values[0]
print(str(lat) + ', ' + str(lon))

59.3417, 18.0549


In [10]:
print(get_point_url(base_url, lon, lat))
point_response = requests.get(get_point_url(base_url, lon, lat))
point_response

https://opendata-download-metfcst.smhi.se/api/category/pmp3g/version/2/geotype/point/lon/18.0549/lat/59.3417/data.json


<Response [200]>

In [14]:
json_point = point_response.json()


In [15]:
ts = json_point['timeSeries']

In [28]:
ts[1]['parameters']

[{'name': 'spp',
  'levelType': 'hl',
  'level': 0,
  'unit': 'percent',
  'values': [0.0]},
 {'name': 'pcat',
  'levelType': 'hl',
  'level': 0,
  'unit': 'category',
  'values': [3]},
 {'name': 'pmin',
  'levelType': 'hl',
  'level': 0,
  'unit': 'kg/m2/h',
  'values': [0.0]},
 {'name': 'pmean',
  'levelType': 'hl',
  'level': 0,
  'unit': 'kg/m2/h',
  'values': [0.0]},
 {'name': 'pmax',
  'levelType': 'hl',
  'level': 0,
  'unit': 'kg/m2/h',
  'values': [0.1]},
 {'name': 'pmedian',
  'levelType': 'hl',
  'level': 0,
  'unit': 'kg/m2/h',
  'values': [0.0]},
 {'name': 'tcc_mean',
  'levelType': 'hl',
  'level': 0,
  'unit': 'octas',
  'values': [8]},
 {'name': 'lcc_mean',
  'levelType': 'hl',
  'level': 0,
  'unit': 'octas',
  'values': [8]},
 {'name': 'mcc_mean',
  'levelType': 'hl',
  'level': 0,
  'unit': 'octas',
  'values': [8.0]},
 {'name': 'hcc_mean',
  'levelType': 'hl',
  'level': 0,
  'unit': 'octas',
  'values': [8]},
 {'name': 't', 'levelType': 'hl', 'level': 2, 'unit': 'C

In [29]:
len(ts)

76